# Python TFM Section

On this section of the TFM I am pretending to do the preparation need for the model, for that:

1. Import all the .csv created on the R section to unify them 
2. Set up a unique data frame where we will have all the variables and information required to the regression model 
3. Compare several models and use the most suitable for this case of study
4. Train and Test the final model and set it up implemented
5. Final conclusions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
#!conda install --yes pathlib 
#$ python -m pip install pathlib

In [3]:
from pathlib import Path

In [4]:
print(Path.cwd())

C:\Users\ES71531200G\Desktop\Data Science\00.TFM


In [5]:
#Defining path to files from the imports folder
%pwd

file1 = "COSTES_E4E_EUROS.csv"
file2 = "LIQUIDACIONES_EUROS.csv"
file3 = "NUCLEAR_WASTES_EUROS.csv"

file5 = "LIQUIDACIONES_MWH.csv"

File_list = [file1, file2, file3]
del(file1,file2,file3,file5)


File_list

['COSTES_E4E_EUROS.csv', 'LIQUIDACIONES_EUROS.csv', 'NUCLEAR_WASTES_EUROS.csv']

In [6]:
%whos

Variable    Type      Data/Info
-------------------------------
File_list   list      n=3
Path        type      <class 'pathlib.Path'>
np          module    <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
pd          module    <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
plt         module    <module 'matplotlib.pyplo<...>\\matplotlib\\pyplot.py'>


Importing directly with the read_csv function retrieves an error due to the encoding used by R during the exportation
The parametres encoding and sep solve the probem

In [7]:
inputpath1 = Path.cwd() / 'Outputs' / 'COSTES_E4E_EUROS.csv' 
df1 = pd.read_csv(inputpath1, sep = ';', header = 0 , encoding = "ISO-8859-1")

I create a df with the same columnames and data types but no rows for using it as the initial frame to append everything alltogheter.

In [8]:
dfTotal = df1[0:0]
del(df1)
dfTotal

,VERSION,ID_UPR,ID_TECNOLOGIA,ID_GRUPO_EMPRESARIAL,ID_AREA_SISTEMA,ID_CONCEPTO_CTRL,VALOR,ID_UNIDAD


In [9]:
path_list = []
for file in File_list:
    inputpath = Path.cwd() / 'Outputs' / file
    print (inputpath)
    df1 = pd.read_csv(inputpath, sep = ';', header = 0 , encoding = "ISO-8859-1")
    dfTotal = dfTotal.append(df1)

C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\COSTES_E4E_EUROS.csv
C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\LIQUIDACIONES_EUROS.csv
C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\NUCLEAR_WASTES_EUROS.csv


C:\Users\ES71531200G\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


The process comes back a warning saying that one column is nmissing in at least one section, so we explore the data to see what's happening

In [10]:
dfTotal.sample(20)

,ID_AREA_SISTEMA,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION
8484,NaN,Banda,EMPRESA1,HN,EUROS,UPR304,3689228,201711
10231,NaN,Terciaria,EMPRESA2,EB,EUROS,UPR367,19355816,201801
3760,NaN,I. R. Cobertura,EMPRESA2,GN,EUROS,UPR322,"-13580,72",201705
3947,ESPAÑA,SERV_GEST_RES,EMPRESA1,NC,EUROS,UPR116,"-8,091671e+06",201808
11045,NaN,M. Diario,EMPRESA3,HN,EUROS,UPR74,107929032,201802
3627,NaN,Terciaria,EMPRESA1,EB,EUROS,UPR2344,"265052,9",201705
934,NaN,M. Diario,EMPRESA1,BP,EUROS,UPR1314,"-98642,25",201702
15702,NaN,M. Intradiario Continuo,EMPRESA1,GN,EUROS,UPR160,-2742308,201809
7509,NaN,I. G. Desvíos y Terciaria,EMPRESA1,EB,EUROS,UPR2344,"-19651,74",201710
1423,NaN,I. G. Desvíos y Terciaria,EMPRESA1,BP,EUROS,UPR2415,"-11942,78",201702


We obserb that the file "LIQUIDACIONES_EUROS" doesn't have the column ID_AREA_SISTEMA.
To overpass this situations, I proceed to replace all the NaNs with the right values through the selection of the unique tuples
[ID_AREA_SISTEMA - ID_UPR].

In [11]:
df_aux = dfTotal[['ID_AREA_SISTEMA', 'ID_UPR']].dropna().drop_duplicates()
df_aux

,ID_AREA_SISTEMA,ID_UPR
0,ESPAÑA,UPR115
5,ESPAÑA,UPR116
10,ESPAÑA,UPR1205
14,ESPAÑA,UPR1206
16,ESPAÑA,UPR1207
20,ESPAÑA,UPR1314
21,ESPAÑA,UPR1315
24,ESPAÑA,UPR160
28,ESPAÑA,UPR162
31,ESPAÑA,UPR1661


And now I will replace the values using a left join with pandas

In [43]:
df_merged = pd.merge(dfTotal, df_aux, on='ID_UPR', how='left')
df_merged.sample(10)
df_merged.columns

Index(['ID_AREA_SISTEMA_x', 'ID_CONCEPTO_CTRL', 'ID_GRUPO_EMPRESARIAL',
       'ID_TECNOLOGIA', 'ID_UNIDAD', 'ID_UPR', 'VALOR', 'VERSION',
       'ID_AREA_SISTEMA_y'],
      dtype='object')

In [44]:
df_merged = df_merged.rename(columns={'ID_AREA_SISTEMA_y': 'ID_AREA_SISTEMA'})
df_merged = df_merged.drop(columns="ID_AREA_SISTEMA_x")
print(df_merged.columns)
df_merged.sample(20)

Index(['ID_CONCEPTO_CTRL', 'ID_GRUPO_EMPRESARIAL', 'ID_TECNOLOGIA',
       'ID_UNIDAD', 'ID_UPR', 'VALOR', 'VERSION', 'ID_AREA_SISTEMA'],
      dtype='object')


,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA
13922,VCF,EMPRESA1,BP,EUROS,UPR1315,"-38233,61",201712,ESPAÑA
16514,G. Desvios,EMPRESA1,CI,EUROS,UPR1862,"707406,3",201803,ESPAÑA
9259,M. Diario,EMPRESA1,GN,EUROS,UPR1850,113913408,201706,PORTUGAL
18126,Res. Pot. Adicional,EMPRESA1,GN,EUROS,UPR2182,4010704,201805,ESPAÑA
9195,VCF,EMPRESA3,GN,EUROS,UPR1714,"699424,3",201706,NaN
15026,Res. Pot. Adicional,EMPRESA1,GN,EUROS,UPR1864,361842,201801,ESPAÑA
9954,I. G. Desvíos y Terciaria,EMPRESA1,CI,EUROS,UPR1661,"-99852,72",201707,ESPAÑA
10314,R. Secundaria,EMPRESA1,LN,EUROS,UPR2342,"-475035,2",201707,ESPAÑA
15170,Res. Pot. Adicional,EMPRESA2,GN,EUROS,UPR2269,353531,201801,NaN
8557,Ajuste,EMPRESA1,CI,EUROS,UPR1863,-3723186,201705,ESPAÑA


In [45]:
df_merged = df_merged[df_merged['ID_GRUPO_EMPRESARIAL'] == 'EMPRESA1']
df_merged.sample(20)

,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA
23447,M. Intradiarios,EMPRESA1,EB,EUROS,UPR2344,-7404435,201812,ESPAÑA
4372,PEAJE GEN,EMPRESA1,GN,EUROS,UPR300,"-1,558844e+04",201809,ESPAÑA
10720,S. Res. Pot. Adicional,EMPRESA1,CI,EUROS,UPR1662,"-1810,4",201708,ESPAÑA
849,LUBRICANTES,EMPRESA1,HN,EUROS,UPR194,"-6,230064e+00",201705,ESPAÑA
2466,OTROS,EMPRESA1,HN,EUROS,UPR304,"-2,105572e+04",201801,ESPAÑA
9042,I. G. Desvíos y Terciaria,EMPRESA1,EB,EUROS,UPR1205,"-20707,5",201706,ESPAÑA
10760,Terciaria,EMPRESA1,BP,EUROS,UPR1752,"535229,8",201708,ESPAÑA
7751,VCF,EMPRESA1,CI,EUROS,UPR1662,"-9479,76",201704,ESPAÑA
22373,Bilateral,EMPRESA1,NC,EUROS,UPR116,1747232727,201811,ESPAÑA
9087,G. Potencia MP,EMPRESA1,BP,EUROS,UPR1315,"235085,8",201706,ESPAÑA


In [46]:
df_merged[df_merged['ID_AREA_SISTEMA'].isna()].head(5)

,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA
5189,D. Medida Contador,EMPRESA1,NC,EUROS,UPR1198,"-35305,63",201701,NaN
5190,S. Res. Pot. Adicional,EMPRESA1,NC,EUROS,UPR1198,"-192,8",201701,NaN
5349,Ajuste,EMPRESA1,BP,EUROS,UPR1751,"-374890,6",201701,NaN
5350,Bilateral,EMPRESA1,BP,EUROS,UPR1751,"-213155,6",201701,NaN
5351,D. Medida Contador,EMPRESA1,BP,EUROS,UPR1751,"-22453,97",201701,NaN


There are still NaN values, but thanks to our knowledge from the original data, we know that there are ONLY 2 UPRs with ID_AREA_SISTEMA = 'Portugal', what means that every NaN value right now should be equal to ESPAÑA, so we replace now all the NANs

In [47]:
df_merged = df_merged.fillna('ESPAÑA')
df_merged.sample(10)

,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA
15981,I. G. Desvíos y Terciaria,EMPRESA1,LN,EUROS,UPR2343,"-277719,3",201802,ESPAÑA
14896,D. Medida Contador,EMPRESA1,BP,EUROS,UPR1751,"-59767,42",201801,ESPAÑA
935,OTROS,EMPRESA1,HN,EUROS,UPR74,"-2,781589e+02",201705,ESPAÑA
19905,M. Intradiarios,EMPRESA1,EB,EUROS,UPR417,-3268295,201807,ESPAÑA
597,COSTE_COMBUSTIBLE,EMPRESA1,GN,EUROS,UPR162,"-4,961761e+05",201704,ESPAÑA
6161,S. Res. Pot. Adicional,EMPRESA1,CI,EUROS,UPR1661,"-57,03",201702,ESPAÑA
6878,Banda,EMPRESA1,BP,EUROS,UPR1315,"86191,18",201703,ESPAÑA
18971,Ajuste,EMPRESA1,EB,EUROS,UPR2344,-1026004,201806,ESPAÑA
9837,M. Intradiarios,EMPRESA1,NC,EUROS,UPR116,"-7554,884",201707,ESPAÑA
21,CANON HID,EMPRESA1,BP,EUROS,UPR1315,"-4,735340e+03",201701,ESPAÑA


In [48]:
df_merged[df_merged['ID_AREA_SISTEMA'].isna()].head(5)

,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA


And finally, reorder de columns to the same order we are already used to 

In [49]:
df_merged = df_merged[['VERSION','ID_UPR','ID_TECNOLOGIA','ID_GRUPO_EMPRESARIAL','ID_AREA_SISTEMA','ID_CONCEPTO_CTRL','VALOR']]
df_merged.reset_index()
df_merged.sample(5)

,VERSION,ID_UPR,ID_TECNOLOGIA,ID_GRUPO_EMPRESARIAL,ID_AREA_SISTEMA,ID_CONCEPTO_CTRL,VALOR
1199,201707,UPR1861,CI,EMPRESA1,ESPAÑA,CENTIMO_VERDE,"-4,673183e+05"
19064,201806,UPR300,GN,EMPRESA1,ESPAÑA,G. Potencia MP,5463842
4961,201812,UPR162,GN,EMPRESA1,ESPAÑA,COSTE_COMBUSTIBLE,"-8,131771e+06"
22505,201811,UPR1662,CI,EMPRESA1,ESPAÑA,Bilateral,98686316
5266,201701,UPR160,GN,EMPRESA1,ESPAÑA,R. Secundaria,"60746,47"


At this point, I will save the current df "df_merged" for the future visualizatin part, this is the structure desired to represent the Integral Margin of the different power plants and so it is for the temporal evolution of every single one of them.

The problem here seemed to be the data types... so first I tried to convert the column value to numeric directly 
with no success...

The error got, suggested that I should convert the data type to floats but the lesson learnt here was that float type in pandas use dots insted of comma for float


In [61]:
df_merged.dtypes

VERSION                  int64
ID_UPR                  object
ID_TECNOLOGIA           object
ID_GRUPO_EMPRESARIAL    object
ID_AREA_SISTEMA         object
ID_CONCEPTO_CTRL        object
VALOR                    int64
dtype: object

In [51]:
df_merged['VALOR'].str.replace(',','.').sample(10)

5257           1045907
15318        -109160.6
15626         16379639
12042         504763.9
235      -1.832035e+06
3946      0.000000e+00
9058            799773
8978           4014358
10331          -314849
18951         -2080.84
Name: VALOR, dtype: object

In [58]:
#pd.to_numeric(df_merged['VALOR'])
df_merged['VALOR'] = pd.to_numeric((df_merged['VALOR'].str.replace(',','.')),errors='coerce').fillna(0).astype(np.int64)
df_merged.sample(10)

,VERSION,ID_UPR,ID_TECNOLOGIA,ID_GRUPO_EMPRESARIAL,ID_AREA_SISTEMA,ID_CONCEPTO_CTRL,VALOR
18376,201805,UPR418,EB,EMPRESA1,ESPAÑA,M. Intradiarios,-1838229
6299,201702,UPR1862,CI,EMPRESA1,ESPAÑA,D. Medida Contador,-159649
5961,201701,UPR74,HN,EMPRESA1,ESPAÑA,G. Potencia MP,68425
7373,201703,UPR2344,EB,EMPRESA1,ESPAÑA,I. G. Desvíos y Terciaria,-391
22216,201810,UPR304,HN,EMPRESA1,ESPAÑA,VCF,13115
6414,201702,UPR2103,EB,EMPRESA1,ESPAÑA,VCF,-26
16375,201803,UPR1661,CI,EMPRESA1,ESPAÑA,Banda,12573729
13006,201711,UPR1314,BP,EMPRESA1,ESPAÑA,I. G. Desvíos y Terciaria,-9447
19628,201807,UPR2141,BP,EMPRESA1,ESPAÑA,Bilateral,-374300
23264,201812,UPR1851,GN,EMPRESA1,PORTUGAL,D. Medida Contador,-354029


In [59]:
df_merged['VALOR'].sum()

176045148550

In [60]:
#df_merged.to_csv?
df_merged.to_csv(Path.cwd() / 'Outputs' / 'INTEGRATED_MARGIN.csv', sep= ';')

Now I proceed to unstack or pivot the table to get the suitable structure for modeling

In [63]:
df_pivoted = df_merged[['VERSION','ID_UPR','ID_TECNOLOGIA','ID_CONCEPTO_CTRL','VALOR']]
#df_pivoted = df_merged.pivot(index=[], columns = 'ID_CONCEPTO_CTRL', values = 'VALOR')
df_pivoted.sample(5)

,VERSION,ID_UPR,ID_TECNOLOGIA,ID_CONCEPTO_CTRL,VALOR
124,201701,UPR2343,LN,COSTE_COMBUSTIBLE,-1209301
2798,201803,UPR1850,GN,COSTE_COMBUSTIBLE,-1108389
17401,201804,UPR2142,BP,S. Res. Pot. Adicional,-8
7640,201704,UPR1207,EB,Ajuste,-229228
14511,201712,UPR300,GN,Ajuste,777635


In [67]:
df_pivoted = df_pivoted.pivot(index=['VERSION', 'ID_UPR', 'ID_TECNOLOGIA'], columns = 'ID_CONCEPTO_CTRL', values = 'VALOR')

ValueError: Length of passed values is 15227, index implies 3

In [64]:
df_prueba = df_pivoted
#df_prueba['VERSION'] = df_prueba['VERSION'].astype(str)
df_prueba.unstack()

VERSION  0         201701
         1         201701
         2         201701
         3         201701
         4         201701
         5         201701
         6         201701
         7         201701
         8         201701
         9         201701
         10        201701
         11        201701
         12        201701
         13        201701
         14        201701
         15        201701
         16        201701
         17        201701
         18        201701
         19        201701
         20        201701
         21        201701
         22        201701
         23        201701
         24        201701
         25        201701
         26        201701
         27        201701
         28        201701
         29        201701
                   ...   
VALOR    23681     350356
         23682    1803102
         23683    1799486
         23684    5044682
         23685    4267657
         23686      52028
         23687    3735746
         236

In [66]:
df_prueba.reset_index().set_index(['VERSION', 'ID_UPR', 'ID_TECNOLOGIA'])['ID_CONCEPTO_CTRL']

VERSION  ID_UPR   ID_TECNOLOGIA
201701   UPR115   NC                    CANON_NC_EST
                  NC               COSTE_COMBUSTIBLE
                  NC                  IMPUESTO ELECT
                  NC                           OTROS
                  NC                   SERV_GEST_RES
         UPR116   NC                    CANON_NC_EST
                  NC               COSTE_COMBUSTIBLE
                  NC                  IMPUESTO ELECT
                  NC                           OTROS
                  NC                   SERV_GEST_RES
         UPR1205  EB                       CANON HID
                  EB                  IMPUESTO ELECT
                  EB                           OTROS
                  EB                  TASAS_MEDIOAMB
         UPR1206  BX                           OTROS
                  BX                  TASAS_MEDIOAMB
         UPR1207  EB                       CANON HID
                  EB                  IMPUESTO ELECT
              

In [10]:
file1 = "COSTES_E4E_EUROS.csv"
file2 = "LIQUIDACIONES_EUROS.csv"
file3 = "NUCLEAR_WASTES_EUROS.csv"

File_list = [file1, file2, file3]
File_list

['COSTES_E4E_EUROS.csv', 'LIQUIDACIONES_EUROS.csv', 'NUCLEAR_WASTES_EUROS.csv']

In [11]:
inputpath1 = Path.cwd() / 'Outputs' / file1
inputpath2 = Path.cwd() / 'Outputs' / file2
inputpath3 = Path.cwd() / 'Outputs' / file3

print (inputpath1)
print (inputpath2)
print (inputpath3)

C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\COSTES_E4E_EUROS.csv
C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\LIQUIDACIONES_EUROS.csv
C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\NUCLEAR_WASTES_EUROS.csv


In [12]:
df1 = pd.read_csv(inputpath1, sep = ';', header = 0 , encoding = "ISO-8859-1")
df2 = pd.read_csv(inputpath2, sep = ';', header = 0 , encoding = "ISO-8859-1")
df3 = pd.read_csv(inputpath3, sep = ';', header = 0 , encoding = "ISO-8859-1")

In [13]:
print(dfTotal.shape)
print(df1.shape)
print(df2.shape)
print(df3.shape)

(23711, 8)
(5166, 8)
(18406, 7)
(139, 8)


In [ ]:
filtrar por emrpesa 1 los valors de liquidaciones 

In [ ]:
crosstab